# 11-7. 프로젝트: 멋진 작사가 만들기

* 자연어 처리에 많이 쓰이는 모듈인 LSTM 을 사용해 문장 생성 모델을 만들어봤다. 학습 데이터는 캐글의 ['Song Lyrics'](https://www.kaggle.com/paultimothymooney/poetry/data) 이다.   
   
   
* .txt 파일 49개를 불러와 전처리를 진행했다. 정규표현식으로 특수문자에 공백을 추가하고, 여러 개의 공백은 하나로 변환하고, 알파벳이나 몇몇 특수문자를 제외한 나머지 문자는 전부 공백으로 변환했다. 이후 ```tf.keras.preprocessing.text.Tokenizer()``` 를 이용해 정제한 가사를 토큰화했다. 토큰화 과정에서 단어 수는 프로젝트 과제에 주어진 **num_words=12000** 로 설정했다.   
   
   
* 각 텐서의 길이를 맞추기 위해 ```tf.keras.preprocessing.sequence.pad_sequences()``` 로 패딩을 설정했다. 프로젝트 과제에 주어진 대로 **maxlen=15**로 설정했다. 여러 번 해본 결과 padding 방향이 pre보다 post일 때 loss가 더 낮아 ```post``` 를 유지했다.   
   
   
* 모델 설계 과정에서 Embedding Layer 의 **embedding_size=512**로, LSTM의 **hidden_size=1024** 로 설정했을 때 **epoch=10**에서 **loss가 2.1089** 로 주어진 조건을 만족했다.   
   
   
* 느낀 점: LSTM은 속도가 느리다.

## 필요한 모듈 import

In [1]:
import glob

import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

## 데이터 읽어오기

In [2]:
txt_file_path = 'data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:50])

데이터 크기: 187088
Examples:
 ['The Cat in the Hat', 'By Dr. Seuss', 'The sun did not shine.', 'It was too wet to play.', 'So we sat in the house', 'All that cold cold wet day.', 'I sat there with Sally.', 'We sat there we two.', 'And I said How I wish', 'We had something to do!', 'Too wet to go out', 'And too cold to play ball.', 'So we sat in the house.', 'We did nothing at all.', 'So all we could do was to', 'Sit!', 'Sit!', 'Sit!', 'Sit!', 'And we did not like it.', 'Not one little bit.', 'BUMP!', 'And then', 'something went BUMP!', 'How that bump made us jump!', 'We looked!', 'Then we saw him step in on the mat!', 'We looked!', 'And we saw him!', 'The Cat in the Hat!', 'And he said to us', 'Why do you sit there like that?', 'I know it is wet', 'And the sun is not sunny.', 'But we can have', 'Lots of good fun that is funny!', 'I know some good games we could play', 'Said the cat.', 'I know some new tricks', 'Said the Cat in the Hat.', 'A lot of good tricks.', 'I will show them to you.',

## 데이터 정제
* preprocess_sentence() 활용
* 문장을 토큰화했을 때 토큰 수가 15개 넘어가면 잘라낼 것

In [3]:
# 전처리
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence) # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>' # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence
    
    
# print(preprocess_sentence("This @_is ;;;sample        sentence.")) # 이 문장이 어떻게 필터링되는지 확인해 보세요.
# print(preprocess_sentence(raw_corpus[9]))

# 정제 데이터 구축
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
    
corpus[:30]

['<start> the cat in the hat <end>',
 '<start> by dr . seuss <end>',
 '<start> the sun did not shine . <end>',
 '<start> it was too wet to play . <end>',
 '<start> so we sat in the house <end>',
 '<start> all that cold cold wet day . <end>',
 '<start> i sat there with sally . <end>',
 '<start> we sat there we two . <end>',
 '<start> and i said how i wish <end>',
 '<start> we had something to do ! <end>',
 '<start> too wet to go out <end>',
 '<start> and too cold to play ball . <end>',
 '<start> so we sat in the house . <end>',
 '<start> we did nothing at all . <end>',
 '<start> so all we could do was to <end>',
 '<start> sit ! <end>',
 '<start> sit ! <end>',
 '<start> sit ! <end>',
 '<start> sit ! <end>',
 '<start> and we did not like it . <end>',
 '<start> not one little bit . <end>',
 '<start> bump ! <end>',
 '<start> and then <end>',
 '<start> something went bump ! <end>',
 '<start> how that bump made us jump ! <end>',
 '<start> we looked ! <end>',
 '<start> then we saw him step in 

In [4]:
# 토큰화
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, # 전체 단어의 개수 
        filters=' ', # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>" # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus) # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus) # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding 메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    6  903 ...    0    0    0]
 [   2  122 2584 ...    0    0    0]
 [   2    6  304 ...    0    0    0]
 ...
 [ 673   27    6 ...    6  189    3]
 [   2  673   27 ...    0    0    0]
 [   2  673   27 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f079092b410>


In [5]:
print(tensor[:3, :]) # length of each tensor: 15

[[   2    6  903   14    6 1350    3    0    0    0    0    0    0    0
     0]
 [   2  122 2584   20    1    3    0    0    0    0    0    0    0    0
     0]
 [   2    6  304  166   70  559   20    3    0    0    0    0    0    0
     0]]


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break
      
len(tokenizer.index_word)

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


27592

In [7]:
src_input = tensor[:, :-1] # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다.
tgt_input = tensor[:, 1:] # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2    6  903   14    6 1350    3    0    0    0    0    0    0    0]
[   6  903   14    6 1350    3    0    0    0    0    0    0    0    0]


## 평가 데이터셋 분리
* sklearn 모듈의 train_test_split() 함수로 훈련/평가 데이터 분리
* 단어장 크기는 12000 이상으로
* 총 데이터의 20%를 평가 데이터셋으로 사용

In [8]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

In [9]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [10]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1
print(VOCAB_SIZE) # 12001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

12001


<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## 인공지능 만들기
* embedding_size 와 hidden_size 를 조절하며 10 epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델 설계

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
# 모델에 데이터 태워보기
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 3.01960572e-05, -3.29620561e-05, -1.31654233e-04, ...,
          2.90282216e-04,  1.32823858e-04,  3.67608009e-05],
        [-1.63750301e-05,  8.19140259e-05, -1.32783316e-04, ...,
          3.16095509e-04, -5.89688498e-05,  4.47397688e-05],
        [-5.14268875e-04,  4.08227468e-04, -1.57484446e-05, ...,
          3.81971331e-04, -6.22258231e-05,  2.24955729e-04],
        ...,
        [-8.65585636e-04,  4.83993528e-04, -1.54074834e-04, ...,
          7.30732572e-04,  6.87194522e-04,  9.69527697e-04],
        [-1.10901624e-03,  8.83469998e-04,  6.39091086e-05, ...,
          6.43138133e-04,  4.21367586e-04,  1.08836498e-03],
        [-1.23321009e-03,  1.49754586e-03,  3.26021487e-04, ...,
          2.09461054e-04,  1.13486014e-04,  1.13339582e-03]],

       [[ 3.01960572e-05, -3.29620561e-05, -1.31654233e-04, ...,
          2.90282216e-04,  1.32823858e-04,  3.67608009e-05],
        [-3.54013056e-04,  1.36320406e-04, -1

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  6144512   
_________________________________________________________________
lstm (LSTM)                  multiple                  6295552   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 33,133,793
Trainable params: 33,133,793
Non-trainable params: 0
_________________________________________________________________


In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10) # loss < 2.2를 충족시킬 것

Epoch 1/10
549/549 [==============================] - 86s 157ms/step - loss: 3.6423
Epoch 2/10
549/549 [==============================] - 90s 164ms/step - loss: 3.1394
Epoch 3/10
549/549 [==============================] - 91s 166ms/step - loss: 2.9492
Epoch 4/10
549/549 [==============================] - 91s 166ms/step - loss: 2.8037
Epoch 5/10
549/549 [==============================] - 91s 165ms/step - loss: 2.6723
Epoch 6/10
549/549 [==============================] - 91s 165ms/step - loss: 2.5501
Epoch 7/10
549/549 [==============================] - 90s 164ms/step - loss: 2.4326
Epoch 8/10
549/549 [==============================] - 90s 164ms/step - loss: 2.3204
Epoch 9/10
549/549 [==============================] - 91s 166ms/step - loss: 2.2132
Epoch 10/10
549/549 [==============================] - 90s 164ms/step - loss: 2.1089


In [16]:
# Q4. 모델이 생성한 가사 한 줄을 제출하세요.
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor) # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you mom , you re my favorite girl <end> '